In [ ]:
import cv2
import numpy as np
import os

In [ ]:
images = []
descs = []
descriptors = np.array([])

for i in os.listdir('output'):
    img = cv2.imread('output/'+i)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(gray, None)
    print(i)
    if des is None:
        continue
    images.append(i)
    descs.append(des)
    if descriptors.shape != (0,): 
        descriptors = np.vstack((descriptors, des))
    else:
        descriptors = des


In [ ]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
desc = np.float32(descriptors)
ret, label, center = cv2.kmeans(desc,200,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

In [ ]:
vecs = []
index = 0
for i in descs:
    temp = np.zeros((200,))
    for j in range(index,index+i.shape[0]):
        temp[label[j]] += 1
    index += len(i)
    vecs.append(temp)

In [ ]:
tfidf = []
ni = np.sum(np.array(vecs),axis=0)
for i in vecs:
    temp = np.zeros((200,))
    #    nd = np.count_nonzero(i)
    nd = np.sum(i)
    for j in range(len(i)):
        temp[j] = (i[j]/nd)*np.log(len(images)/ni[j])
    temp = temp / np.linalg.norm(temp)
    tfidf.append(temp)

In [ ]:
from scipy import spatial
tree = spatial.KDTree(center)
input_image = cv2.imread('input/input.png')
gray_input = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
kp, input_descriptors = sift.detectAndCompute(gray_input, None)
input_vec = np.zeros((200,))
for i in input_descriptors:
    j = tree.query(i)
    input_vec[j[1]] += 1
input_tfidf = np.zeros((200,))
for i in range(len(input_vec)):
    input_tfidf[i] = (input_vec[i]/np.sum(input_vec))*np.log(len(images)/ni[i])
input_tfidf = input_tfidf / np.linalg.norm(input_tfidf)

    
    


In [ ]:
min_dot = []
for i in tfidf:
    min_dot.append(np.dot(i,input_tfidf))
min_dot = np.array(min_dot)
print(min_dot.argsort()[-3:][::-1])
for i in min_dot.argsort()[-3:][::-1]:
    print(images[i])